# Matrix Multiply

## numpy

In [102]:
import numpy as np

n = 2000
A = np.random.rand(n, n).astype(np.single)
B = np.random.rand(n, n).astype(np.single)

%timeit -n1 -r1 A * B
A = B = None

1 loop, best of 1: 3.96 ms per loop


## pytorch

### cpu

In [59]:
import torch

n = 2000
A = torch.rand(n, n, dtype = torch.float, device = "cpu")
B = torch.rand(n, n, dtype = torch.float, device = "cpu")

%timeit -n1 -r1 (A * B).numpy()
A = B = None

1 loop, best of 1: 4.18 ms per loop


### cuda

In [98]:
import torch

n = 2000
A = torch.rand(n, n, dtype = torch.float, device = "cuda")
B = torch.rand(n, n, dtype = torch.float, device = "cuda")

%timeit -n1 -r1 (A * B).cpu().numpy()
A = B = None

1 loop, best of 1: 5.09 ms per loop


## tensorflow

In [48]:
import tensorflow as tf

n = 2000
A = tf.random.uniform((n, n), dtype = tf.dtypes.float32)
B = tf.random.uniform((n, n), dtype = tf.dtypes.float32)

%timeit -n1 -r1 (A * B).numpy()
A = B = None

1 loop, best of 1: 21 ms per loop


## blas

In [88]:
%%file mm.cpp
#include <cblas.h>
#include <random>
#include <chrono>
#include <iostream>
#include <memory>

using namespace std;
using namespace chrono;

int main()
{
    static default_random_engine rng(1234);
 
    auto random_floats = [] (size_t n)
    {
        static normal_distribution<float> normal;
        auto a = new float[n];
     
        for (size_t i = 0; i < n; ++i)
        {
            a[i] = normal(rng);
        }

        return a;
    };

    size_t m = 2000;
    size_t n = 2000;
    size_t k = 2000;

    unique_ptr<float[]> A(random_floats(m * n));
    unique_ptr<float[]> B(random_floats(n * k));
    unique_ptr<float[]> C(random_floats(m * k));
 
    // C = A * B;
    auto start = system_clock::now();

    cblas_sgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, m, n, k, 1, A.get(), m, B.get(), k, 0, C.get(), m);
    cout << duration_cast<milliseconds>(system_clock::now() - start).count() << " ms " << endl;
}

Overwriting mm.cpp


In [89]:
!g++ mm.cpp -lblas -o mm && ./mm

225 ms 


## cublas

In [90]:
%%file mm.cu
#include "cublas_v2.h"
#include <curand.h>
#include <chrono>
#include <iostream>

using namespace std;
using namespace chrono;

int main()
{
    curandGenerator_t rng;
    curandCreateGenerator(&rng, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(rng, 1234);

    cublasHandle_t handle; 
    cublasCreate(&handle);

    size_t m = 2000;
    size_t n = 2000;
    size_t k = 2000;

    float *d_A, *d_B, *d_C;
 
    cudaMalloc(&d_A, m * n * sizeof(*d_A));
    cudaMalloc(&d_B, n * k * sizeof(*d_B));
    cudaMalloc(&d_C, m * k * sizeof(*d_C));

    curandGenerateUniform(rng, d_A, m * n);
    curandGenerateUniform(rng, d_B, n * k);

    // C = A * B;
    float alpha = 1;
    float beta = 0;
    auto start = system_clock::now();

    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, &alpha, d_A, m, d_B, k, &beta, d_C, m);
    cout << duration_cast<microseconds>(system_clock::now() - start).count() << " μs " << endl;

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

Overwriting mm.cu


In [91]:
!nvcc -arch=sm_75 mm.cu -lcublas -lcurand -o mm && ./mm

152 μs 


In [ ]:
!nvprof ./mm

## eigen

In [ ]:
!wget https://gitlab.com/libeigen/eigen/-/archive/3.3.8/eigen-3.3.8.tar.bz2
!tar xvjf eigen-3.3.8.tar.bz2
!mv eigen-3.3.8/Eigen /usr/local/include/.

In [86]:
%%file mm.cpp
#define EIGEN_USE_BLAS
#include <Eigen/Dense>
#include <chrono>
#include <iostream>

using namespace std;
using namespace chrono;
using namespace Eigen;

int main()
{
    size_t m = 2000;
    size_t n = 2000;
    size_t k = 2000;

    auto A = MatrixXf::Random(m, n);
    auto B = MatrixXf::Random(n, k);
    auto start = system_clock::now();
    auto C = A * B;

    C.eval();
    cout << duration_cast<milliseconds>(system_clock::now() - start).count() << " ms " << endl;
}

Overwriting mm.cpp


In [87]:
!g++ mm.cpp -lblas -o mm && ./mm

595 ms 


## pytorch c++

In [ ]:
!wget https://download.pytorch.org/libtorch/cu102/libtorch-cxx11-abi-shared-with-deps-1.7.1.zip
!unzip libtorch-cxx11-abi-shared-with-deps-1.7.1.zip
!rm libtorch-cxx11-abi-shared-with-deps-1.7.1.zip
!cp -r libtorch/include /usr/local/.
!cp -r libtorch/include/torch/csrc/api/include/torch /usr/local/include/.
!cp libtorch/lib/*.so* /usr/lib/x86_64-linux-gnu/.
!rm -rf libtorch

In [ ]:
%%file mm.cpp
#include <torch/torch.h>
#include <iostream>

using namespace std;

int main()
{
    cout << "hello" << endl;
    torch::Tensor tensor = torch::eye(3);
    cout << tensor << endl;
}

In [ ]:
!g++ mm.cpp -ltorch -ltorch_cpu -lc10 -o mm
!./mm

## julia

In [ ]:
%%shell
wget https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz
tar -x -f julia-1.6.0-linux-x86_64.tar.gz -C /usr/local --strip-components 1

In [ ]:
!julia -e 'using Pkg; Pkg.add("CUDA")'

In [77]:
%%file mm.jl
using CUDA

A = rand(Float32, (2000, 2000))
A * A
@time A * A

A = CuArray(A)
A * A
@time A * A

Overwriting mm.jl


In [78]:
!julia mm.jl

  0.187886 seconds (2 allocations: 15.259 MiB)
  0.000417 seconds (72 allocations: 1.516 KiB)
